In [6]:
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 
# 定义铜的晶格常数（单位：Å）
lattice_constant = 3.61

# 定义铜的FCC晶胞的基矢量
lattice_vectors = (Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))'


# 定义原子类型
struct Atom
    element::String
    position::Vector{Float64}
    cn::Int
    function Atom(element,position,cn)
        new(element,position,cn)
    end
    function Atom(element,position)
        new(element,position,1)
    end
end


# 定义晶胞类型
struct UnitCell
    lattice_vectors::Matrix{Float64}
    atoms::Vector{Atom}
    copy::Vector{Int}
    Volume::Float64
    function UnitCell(lattice_vectors::Matrix{Float64}, atoms::Vector{Atom}, copy::Vector{Int})
        v=copy[1]*copy[2]*copy[3]*det(lattice_vectors)
        new(lattice_vectors, atoms,copy,v)
    end
    function UnitCell(lattice_vectors::Matrix{Float64}, atoms::Vector{Atom})
        new(lattice_vectors, atoms, Vector([1,1,1]))
    end
    function UnitCell(lattice_vectors::Adjoint{Float64, Matrix{Float64}}, atoms::Vector{Atom})
        new(lattice_vectors, atoms, Vector([1,1,1]))
    end
end

function cell_energy(cell::UnitCell,cut_off::Float64=10.0)
    energy=0.0
    for i in 1:length(cell.atoms)
        for j in i+1:length(cell.atoms)
            energy+=atoms_energe(cell.atoms[i],cell.atoms[j],cell.lattice_vectors,cut_off)
        end
    end
    return energy
end

function copycell(cell::UnitCell,a::Int,b::Int,c::Int,tol::Float64=0.001)::UnitCell
    atoms=Vector{Atom}([])
    pl=Vector{Vector{Float64}}([])
    for atom in cell.atoms
        for i in 0:a-1
            for j in 0:b-1
                for k in 0:c-1
                    newp=atom.position+Vector([i,j,k])
                    if newp in pl
                        continue
                    end
                    if (newp==[0.0,0.0,0.0]||newp==[1.0*a,1.0*b,1.0*c])
                        cn=8
                    elseif count(x -> abs(x) < tol, newp)==2
                        cn=4
                    elseif count(x -> abs(x) < tol, newp)==1
                        cn=2
                    else
                        cn=1 
                    end

                    push!(atoms,Atom(atom.element,newp,cn))
                    push!(pl,newp)
                end
            end
        end      
    end
    
    return UnitCell(cell.lattice_vectors,atoms,Vector([a,b,c]))
end


# 定义一个函数来可视化晶胞内的原子
function visualize_unitcell_atoms(cell::UnitCell,atomcolor=:red,markersize=10,veccolor=:blue,linewith=0.1)::Figure
    fig =GLMakie.Figure(size = (800, 600))
    ax = GLMakie.Axis3(fig[1, 1], title = "Visualization of Atoms in the Unit Cell", 
               xlabel = "X", ylabel = "Y", zlabel = "Z")
    M=cell.lattice_vectors
    for atom in cell.atoms
        p=M*atom.position
        GLMakie.scatter!(ax,p..., color = atomcolor, markersize = markersize)
    end
    
    # 绘制晶格向量（晶胞的边缘）
    origin=GLMakie.Point3f0(0,0,0)
    for (k,vec) in enumerate(eachcol(cell.lattice_vectors))
        vc=GLMakie.Point3f0(vec*cell.copy[k])
        GLMakie.arrows!(ax, [origin], [origin + vec], color = veccolor, linewidth = linewith)
    end
    rg=maximum(cell.lattice_vectors*cell.copy)
    xlims!(ax, 0, rg)
    ylims!(ax, 0, rg)
    zlims!(ax, 0, rg)
    return fig
end

visualize_unitcell_atoms (generic function with 5 methods)

In [7]:
vc=Vector([0.0,0.0,0.0])
vc==[0.0,0.0,0.0]

true

In [8]:
# 定义一个泛型结构体 Interaction
struct Interaction{F1, F2, F3, F4}
    energe::F1  # 势能函数
    force::F2   # 力函数
    cutenerge::F3  # 截断势能函数
    cutforce::F4   # 截断力函数
    cutoff::Float64  # 截断距离
    cutrg::Float64   # 截断范围


    # 定义构造函数
    function Interaction(energe::F1, force::F2, cutoff::Float64, cutrg::Float64) where {F1, F2}
        

        # 初始化截断势能和力的参数
        Er1 = 0
        dU1 = 0
        Er2 = energe(cutoff - cutrg)
        dU2 = (force(Vector([cutoff - cutrg,0,0])))[1]
        a = -(dU2 - dU1) / (2 * cutrg)
        b = -2 * a * (cutoff )
        c = Er1 - a * (cutoff)^2 - b * (cutoff)

        # 定义截断势能函数
        function cutenerge(r::Float64)
            nr = r
            if nr > cutoff
                return 0.0
            elseif nr < cutoff - cutrg
                return energe(r)
            else
                return a * nr^2 + b * nr + c
            end
        end

        # 定义截断力函数，接收向量输入
        function cutforce(r::Vector{Float64})
            nr = norm(r)
            if nr > cutoff
                return Vector(zeros(3))
            elseif nr < cutoff - cutrg
                return force(r)
            else
                return ((2 * a * nr + b) / nr) * r
            end
        end

        # 定义截断力函数的重载，使其可以接收 Float64 类型的输入
        cutforce(r::Float64) = (cutforce(Vector([r, 0, 0])))[1]

        # 返回新的 Interaction 实例
        new{F1, F2, typeof(cutenerge), typeof(cutforce)}(energe, force, cutenerge, cutforce, cutoff, cutrg)
    end
end


In [9]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 3.61

# 定义铜的FCC晶胞的基矢量
lattice_vectors = (Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))'

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom("Cu", pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,2,2,1)
fig=visualize_unitcell_atoms(cpcell);

In [10]:
cpcell

UnitCell([3.61 0.0 0.0; 0.0 3.61 0.0; 0.0 0.0 3.61], Atom[Atom("Cu", [0.0, 0.0, 0.0], 8), Atom("Cu", [0.0, 1.0, 0.0], 4), Atom("Cu", [1.0, 0.0, 0.0], 4), Atom("Cu", [1.0, 1.0, 0.0], 2), Atom("Cu", [0.0, 0.5, 0.5], 2), Atom("Cu", [0.0, 1.5, 0.5], 2), Atom("Cu", [1.0, 0.5, 0.5], 1), Atom("Cu", [1.0, 1.5, 0.5], 1), Atom("Cu", [0.5, 0.0, 0.5], 2), Atom("Cu", [0.5, 1.0, 0.5], 1)  …  Atom("Cu", [0.5, 0.5, 1.0], 1), Atom("Cu", [0.5, 1.5, 1.0], 1), Atom("Cu", [1.5, 0.5, 1.0], 1), Atom("Cu", [1.5, 1.5, 1.0], 1), Atom("Cu", [2.0, 0.0, 1.0], 2), Atom("Cu", [2.0, 1.0, 1.0], 1), Atom("Cu", [2.0, 2.0, 0.0], 2), Atom("Cu", [0.0, 2.0, 1.0], 2), Atom("Cu", [1.0, 2.0, 1.0], 1), Atom("Cu", [2.0, 2.0, 1.0], 8)], [2, 2, 1], 188.18352399999998)

In [63]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)  
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return 24*(2/rn^14-1/rn^8)*r
end


interaction = Interaction(lj, Flj, 3.0, 2.0)

x=0.8:0.01:5
y=interaction.cutforce.(x)
lines(x,y)

```
count_zeros(x::Integer) -> Integer
```

Number of zeros in the binary representation of `x`.

# Examples

```jldoctest
julia> count_zeros(Int32(2 ^ 16 - 1))
16

julia> count_zeros(-1)
0
```


In [23]:
vec = [1e-5, 0.01, -0.001, 0.02, -1e-6]  # 示例向量
tol = 0.001  # 设置阈值

num_small_elements = count(x -> abs(x) < tol, vec)  # 计算绝对值小于eps的元素个数
println(num_small_elements)



2
